# Imports

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import src.utils.preprocessing as pre
import src.utils.plotting as plot
import src.models.sentiment_analysis as sa
import src.models.topics_extraction as te
import src.models.predictions as pred
import src.models.cooperation as coop

# Data and Pre-Processing
### Part 1 EDA

In [14]:
data = pre.import_RFA()
print(data.shape)
data.head(5)

(198275, 7)


,SRC,TGT,VOT,RES,YEA,DAT,TXT
0,Steel1943,BDD,1,1,2013,"23:13, 19 April 2013",Support as conom.
1,Cuchullain,BDD,1,1,2013,"01:04, 20 April 2013",Support as nominator.
2,INeverCry,BDD,1,1,2013,"23:43, 19 April 2013",Support per noms.
3,Cncmaster,BDD,1,1,2013,"00:11, 20 April 2013",Support per noms. BDD is a strong contributor ...
4,Miniapolis,BDD,1,1,2013,"00:56, 20 April 2013","Support, with great pleasure. I work with BDD ..."


In [79]:
# df_processed, data_eda, unique_elections, unique_candidate_freq_table, single_runners_list, multiple_runners_list, flagged_elec_id = pre.preprossess_eda(data)
# df1 = data_eda.copy()
# votes, df1 = pre.get_votes(df1)

# Sentiment analysis and Topics Extraction
### Running the SA and Topics Extraction

In [2]:
dfsa, qs = pre.complete_prepro_w_sa_topics()

In [3]:
import pandas as pd

In [4]:
# data = pre.merge_sa_eda(data_eda,dfsa)
data = pd.read_csv('res/data/merged_id_sa.csv')

In [5]:
plot.plot_sentiment_byPass(data,prt=True)

In [ ]:
plot.plot_sentiment_byPass(data,vader=True,prt=True)

In [7]:
plot.plot_sentiments_byYear(data,prt=True)

In [ ]:
plot.plot_sentiments_byYear(data,vader=True,prt=True)